Point of Sales

In [1]:
from geopy.geocoders import Nominatim
import time
from pprint import pprint
import pandas as pd
import numpy as np
import time

In [44]:
cities = pd.read_csv('Data/worldcities.csv')

In [45]:
#Keep only useful columns
cities = cities[['city', 'lat', 'lng', 'country', 'population']]

#Keep only information about cities that have over 50,000 inhabithants
cities = cities[cities.population >= 50000]

cities

,city,lat,lng,country,population
0,Tokyo,35.6897,139.6922,Japan,37977000.0
1,Jakarta,-6.2146,106.8451,Indonesia,34540000.0
2,Delhi,28.6600,77.2300,India,29617000.0
3,Mumbai,18.9667,72.8333,India,23355000.0
4,Manila,14.6000,120.9833,Philippines,23088000.0
...,...,...,...,...,...
12860,Sardasht,36.1553,45.4789,Iran,50000.0
12861,Buluan,6.7154,124.7854,Philippines,50008.0
12864,Shawinigan,46.5667,-72.7500,Canada,50060.0
12867,Kahemba,-7.2829,19.0000,Congo (Kinshasa),50000.0


In [46]:
#Define lists of companies from different sectors and their type
industries_mode = ["Nike", "Zara", "Adidas", "H & M", "Uniqlo", "Fila", "Gap", "Petit Bateau", "Reebok", "Aigle", "Benetton", "Celio", "Diesel", "Polo Ralph Lauren", "Puma", "Quiksilver",  "Lacoste",  "Pull & Bear", "Jennyfer"]
industries_cosmetic = ["MAC", "Dior", "L'Oréal", "Clinique", "Maybelline", "Nars", "Laura Mercier", "Sephora", "Yves Saint Laurent", "Covergirl"]
industries_luxury = ["Louis Vuitton", "Cartier", "Ray Ban", "Gucci", "Rolex", "Coach", "Hermès", "Prada", "Swaroski", "Hugo Boss", "Giorgio Armani", "Mickael Kors"]
industries_auto = ["Volkswagen", "Toyota", "Renault", "Nissan", "Mitsubishi", "Daimler", "Ford", "Tesla", "General Motors", "Honda", "Fiat", "BWM", "Peugeot", "Mclaren", "Alpha Romeo", "Aston Martin", "BMW", "Mercedes-Benz", "Mini", "Ferrari"]
industries_tech = ["Apple", "Microsoft", "Canon", "Samsung Electronics", "Alphabet", "AT&T", "Amazon", "Verizon Communication", "Facebook", "Alibaba", "Intel", "Softbank", "IBM", "Oracle", "SAP", "Orange", "Accenture", "Qualcomm", "Paypal", "HP", "Philips", "Adobe", "NVIDIA", "Vodafone", "Vivendi", "Capgemini", "Nokia", "Rakuten"]
all_industries = industries_mode + industries_cosmetic + industries_luxury + industries_auto + industries_tech
type_industries = ["Mode"] * len(industries_mode) + ["Cosmetic"] * len(industries_cosmetic) + ['Luxury'] * len(industries_luxury) + ['Car'] * len(industries_auto) + ["Tech"] * len(industries_tech)

company_db = pd.DataFrame({"Company Name": all_industries, "Type": type_industries}) #Transform the companies information into a Dataframe
company_db.head(5)

,Company Name,Type
0,Nike,Mode
1,Zara,Mode
2,Adidas,Mode
3,H & M,Mode
4,Uniqlo,Mode


In [ ]:
lst_company, lst_type_comp, lst_country, lst_city, lst_pop, lst_latitude, lst_longitude = [], [], [], [], [], [], []
ini_time = time.time()
for comp, type in zip(company_db['Company Name'], company_db['Type']):
    print(round(time.time()-ini_time), comp, type, len(lst_city))
    ini_time = time.time()
    start_time = time.time()
    count = 0
    for cit, countr, popul in zip(cities.city, cities.country, cities.population): 
        try:
            loc= comp + "," + cit

            #making an instance of Nominatim class
            geolocator_city = Nominatim(user_agent="my_request")
            
            #applying geocode method to get the location
            location = geolocator_city.geocode(loc)
            
            #printing address and coordinates
            lst_latitude.append(location.latitude)
            lst_longitude.append(location.longitude)
            lst_company.append(comp)
            lst_type_comp.append(type)
            lst_country.append(countr)
            lst_city.append(cit)
            lst_pop.append(popul)
            start_time = time.time()
            count +=1
            
        except Exception:
            if  time.time() - start_time > 60:
                break
            else:
                pass
 
pos_db = pd.DataFrame({
    "Company": lst_company,
    "Type": lst_type_comp,
    "Country": lst_country,
    "City": lst_city,
    "Population": lst_pop,
    "Latitude": lst_latitude,
    "Longitude": lst_longitude,
})

pos_db.to_csv("Data/point_of_sale_db.csv")

In [44]:
pos_db[['Company', 'Type', 'Country', 'City', 'Population', 'Latitude', 'Longitude']].head()

,Unnamed: 0,Unnamed: 0.1,Company,Type,Country,City,Population,Latitude,Longitude
0,0,0,Nike,Mode,Japan,Tokyo,37977000.0,35.705617,139.576935
1,1,1,Nike,Mode,Indonesia,Jakarta,34540000.0,-6.265635,106.783054
2,2,2,Nike,Mode,India,Delhi,29617000.0,28.557346,77.163798
3,3,3,Nike,Mode,India,Mumbai,23355000.0,19.136268,72.829615
4,4,4,Nike,Mode,Philippines,Manila,23088000.0,14.652022,121.075537


In [16]:
print('Number of POS:', len(pos_db))

Number of POS: 5213


In [29]:
print('Number of Countries:', len(np.unique(pos_db['Country'])))

Number of Countries: 82


In [28]:
print('Number of Cities:', len(np.unique(pos_db['City'])))

Number of Cities: 422


In [30]:
print('Number of Companies:', len(np.unique(pos_db['Company'])))

Number of Companies: 80


In [31]:
print('Type of Companies:', set(pos_db['Type']), "-", len(np.unique(pos_db['Type'])))

Type of Companies: {'Car', 'Cosmetic', 'Luxury', 'Tech', 'Mode'} - 5
